<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/Copy-baseline_Eval_NLP801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets nltk sentencepiece tqdm

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm  # Import tqdm for the progress bar
import re  # Import the re module for regular expressions
from nltk.translate.bleu_score import SmoothingFunction

# Initialize the tokenizer and fine-tuned T5-small model
tokenizer = T5Tokenizer.from_pretrained("HamdanXI/t5_small_aslg_pc12")
model = T5ForConditionalGeneration.from_pretrained("HamdanXI/t5_small_aslg_pc12")

# Load the ASLG-PC12 dataset
dataset = load_dataset("aslg_pc12")

smoother = SmoothingFunction()

# Modify the copy baseline function to just normalize and return the gloss text
def copy_baseline(input_text):
    # Convert to lowercase
    baseline_text = input_text.lower()

    # Normalize whitespace (replace consecutive spaces with a single space)
    baseline_text = re.sub(r'\s+', ' ', baseline_text).strip()

    return baseline_text

baseline_bleu_scores = []

# Let's print the first 20 (or another number of your choice) comparisons
num_to_print = 20
print_count = 0

# Create a tqdm progress bar for the dataset
for example in tqdm(dataset["train"]):  # Use tqdm to wrap the dataset iteration
    gloss_text = example["gloss"]
    reference_text = example["text"]

    # Generate text using the copy baseline
    baseline_text = copy_baseline(gloss_text)  # No need for the prefix

    # Calculate BLEU score for the baseline
    baseline_bleu = sentence_bleu([reference_text.split()], baseline_text.split(), smoothing_function=smoother.method1)

    # Append BLEU score to the list
    baseline_bleu_scores.append(baseline_bleu)

    # If you want to inspect some of the results:
    if print_count < num_to_print:
        print("Gloss Text       :", gloss_text)
        print("Generated Text   :", baseline_text)
        print("Reference Text   :", reference_text)
        print("BLEU Score       :", baseline_bleu)
        print("------------------------------")
        print_count += 1

# Calculate the average BLEU score for the baseline
average_baseline_bleu = sum(baseline_bleu_scores) / len(baseline_bleu_scores)

# Print the final comparison (including the case-insensitive BLEU score)
print("Average Case-Insensitive BLEU Score (Copy Baseline):", average_baseline_bleu)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 111/87710 [00:00<01:24, 1031.68it/s]

Gloss Text       : ﻿MEMBERSHIP PARLIAMENT SEE MINUTE

Generated Text   : ﻿membership parliament see minute
Reference Text   : ﻿membership of parliament see minutes

BLEU Score       : 0.14643803168582153
------------------------------
Gloss Text       : APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE

Generated Text   : approval minute desc-previous sit see minute
Reference Text   : approval of minutes of previous sitting see minutes

BLEU Score       : 0.03478700554542394
------------------------------
Gloss Text       : MEMBERSHIP PARLIAMENT SEE MINUTE

Generated Text   : membership parliament see minute
Reference Text   : membership of parliament see minutes

BLEU Score       : 0.14643803168582153
------------------------------
Gloss Text       : VERIFICATION CREDENTIALS SEE MINUTE

Generated Text   : verification credentials see minute
Reference Text   : verification of credentials see minutes

BLEU Score       : 0.14643803168582153
------------------------------
Gloss Text       : DO

100%|██████████| 87710/87710 [00:22<00:00, 3876.57it/s]

Average Case-Insensitive BLEU Score (Copy Baseline): 0.18673767129946803
